# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [24]:
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'iEG3V3LLXNerRncmEWgv8lQ4n'
    CONSUMER_SECRET ='67HGDVQRF7YxJ6ZUFnUlvwreR2CShhao1Zx6k5wo0OoRRkGez4'
    OAUTH_TOKEN = '2380166015-sZywqhm3U9naQ154u0qQQzBv1FJgAWDVSWN6ROB'
    OAUTH_TOKEN_SECRET = '1AQCSvqp06DTpwYbgBXooWVmLuzWJ8zpsFpWgaazUhhha'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import json

# login to api
twitter_api = oauth_login()

# set initial search
search_results = twitter_api.search.tweets(q='Trump', count=100)
statuses = search_results['statuses']
print("Length of statuses", len(statuses))
max_tweets = 1000

while(len(statuses) < max_tweets):
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    # search api will take the key words dictionary
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    print("Length of statuses", len(statuses))

# Write the status to a local file
outfile = open('tweets.txt', 'w')
outfile.write(json.dumps(statuses, indent=1))
outfile.close()
print "Tweets output to file successfully!"









('Length of statuses', 100)
('Length of statuses', 200)
('Length of statuses', 300)
('Length of statuses', 400)
('Length of statuses', 500)
('Length of statuses', 600)
('Length of statuses', 700)
('Length of statuses', 800)
('Length of statuses', 900)
('Length of statuses', 1000)
Tweets output to file successfully!


### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [25]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import time
from collections import Counter
from prettytable import PrettyTable

# Collect status texts
status_texts = [ status['text'] 
                 for status in statuses ]

# Use word to store all words in tweets and use counter to do analysis
word = []
for text in status_texts:
    for w in text.split():
        word.append(w)
word_freq = Counter(word)

# Put the data into the prettytable
top_word = PrettyTable(field_names=['Word','Count']) # Header
for item in word_freq.most_common()[:30]:
    top_word.add_row(item) # Adding rows
top_word.align['Word'], top_word.align['Count'] = 'c', 'r' # Set column alignment
print top_word














+----------------------+-------+
|         Word         | Count |
+----------------------+-------+
|          RT          |   757 |
|        Trump         |   629 |
|          to          |   316 |
|          of          |   281 |
|          a           |   274 |
|         the          |   258 |
|          is          |   242 |
|         for          |   213 |
|         and          |   138 |
|          in          |   133 |
|        Donald        |   129 |
|      supporters      |   103 |
|          on          |    91 |
|       Clinton        |    88 |
|       Hillary        |    88 |
|         are          |    86 |
|          I           |    84 |
|         you          |    79 |
|          at          |    72 |
|         that         |    68 |
|         his          |    67 |
|         this         |    65 |
|        about         |    62 |
|        money         |    61 |
| #BasketOfDeplorables |    60 |
|        &amp;         |    58 |
|         with         |    56 |
|       Tr

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [26]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect all retweets
retweets = []
for status in statuses:
    if 'retweeted_status' in status:
        retweets.append((status['retweet_count'], status['retweeted_status']['user']['screen_name'], status['text']))

# Generate the pretty table
most_retweet = PrettyTable(field_names=['Count', 'Screen Name', 'Tweet Text']) # Header
for item in sorted(retweets, reverse=True)[:10]:
    most_retweet.add_row(item) # Adding rows
most_retweet.align['Count'], most_retweet.align['Screen Name'], most_retweet.align['Tweet Text'] = 'l', 'l', 'l'
most_retweet.max_width['Tweet Text'] = 70 # Set max width for "Tweet Text" column
print most_retweet











+-------+---------------+------------------------------------------------------------------------+
| Count | Screen Name   | Tweet Text                                                             |
+-------+---------------+------------------------------------------------------------------------+
| 30293 | MrJamesonNeat | RT @MrJamesonNeat: Just a reminder Donald Trump's upcoming case for    |
|       |               | raping a 13 year old aspiring model is scheduled for October 14th.     |
|       |               | http…                                                                  |
| 22896 | cocochanelite | RT @cocochanelite: "if Trump wins I'm moving to Canada"                |
|       |               | ok so ppl fleeing their country from political crisis isn't ok unless  |
|       |               | it's USA htt…                                                          |
| 22896 | cocochanelite | RT @cocochanelite: "if Trump wins I'm moving to Canada"                |
|       | 

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [27]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Collect hashtags
hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Collect mention user
mention_user = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

# Generate pretty table
for label, data in (('Hashtag',hashtags), ('Mention User', mention_user)): # Access two table data
    table = PrettyTable(field_names=[label, 'Count']) # Header
    count = Counter(data) # Count the data
    for item in count.most_common()[:10]:
        table.add_row(item) # Adding each row
    table.align[label], table.align['Count'] = 'l', 'c'
    print table






+-----------------------+-------+
| Hashtag               | Count |
+-----------------------+-------+
| BasketOfDeplorables   |   78  |
| Trump                 |   58  |
| Trump2016             |   20  |
| MAGA                  |   17  |
| Hillary               |   13  |
| basketofdeplorabes    |   12  |
| NeverTrump            |   11  |
| trump                 |   11  |
| MakeAmericaGreatAgain |   9   |
| p2                    |   9   |
+-----------------------+-------+
+-----------------+-------+
| Mention User    | Count |
+-----------------+-------+
| realDonaldTrump |   57  |
| HillaryClinton  |   38  |
| Fahrenthold     |   23  |
| owillis         |   21  |
| JuddLegum       |   16  |
| thehill         |   15  |
| mitchellvii     |   13  |
| zerohedge       |   11  |
| ScottPresler    |   10  |
| AmericanMex067  |   10  |
+-----------------+-------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [75]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Get all friends
friends = twitter_api.friends.list(screen_name='realDonaldTrump')
friend_list = friends['users']
next_cursor = friends['next_cursor']
while (next_cursor != -1 and len(friend_list) < 2000): # Navigate through pages by cursor and rate controller
    friends = twitter_api.friends.list(screen_name='realDonaldTrump', cursor=next_cursor, count=200)
    friend_list += friends['users']
    next_cursor = friends['next_cursor']


# Get all followers
followers = twitter_api.followers.list(screen_name='realDonaldTrump')
follower_list = followers['users']
next_cursor = followers['next_cursor']
rate_counter = 1
while (next_cursor != -1 and len(follower_list) < 2000): # Navigate through pages by cursor and rate controller
    followers = twitter_api.followers.list(screen_name='realDonaldTrump', cursor=next_cursor, count=200)
    follower_list += followers['users']
    next_cursor = followers['next_cursor']




In [77]:
# Create table for friend
friend_table = PrettyTable(field_names=['Friend ID', 'Screen Name']) # Header
for user in friend_list[:20]:
    friend_table.add_row([user['id'], user['screen_name']])
friend_table.align['Friend ID'], friend_table.align['Screen Name'] = 'l', 'c' # Alignment
print friend_table

# Create table for follower
follower_table = PrettyTable(field_names=['Follower ID', 'Screen Name']) # Header
for user in follower_list[:20]:
    follower_table.add_row([user['id'], user['screen_name']])
follower_table.align['Follower ID'], follower_table.align['Screen Name'] = 'l', 'c' # Alignment
print follower_table

+--------------------+-----------------+
| Friend ID          |   Screen Name   |
+--------------------+-----------------+
| 34077321           | BigDawgSwifty13 |
| 261014695          |     LGlick1     |
| 720293443260456960 |      Trump      |
| 2325495378         |    TrumpGolf    |
| 245963716          |  TiffanyATrump  |
| 50769180           |  IngrahamAngle  |
| 22203756           |    mike_pence   |
| 729676086632656900 |    TeamTrump    |
| 14669951           |  DRUDGE_REPORT  |
| 475802156          | MrsVanessaTrump |
| 75541946           |   LaraLeaTrump  |
| 41634520           |   seanhannity   |
| 37764422           |    foxnation    |
| 4121225056         |  CLewandowski_  |
| 196168350          |    AnnCoulter   |
| 2908170952         |  DiamondandSilk |
| 34852681           |  KatrinaCampins |
| 25429371           |  KatrinaPierson |
| 52136185           | MichaelCohen212 |
| 15513604           |  foxandfriends  |
+--------------------+-----------------+
+---------------

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [78]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Create Set for friend and follower
friend = set([(friend['id'], friend['screen_name']) for friend in friend_list])
follower = set([(follower['id'], follower['screen_name']) for follower in follower_list])

# Check mutual friend
mutual_friend = friend.intersection(follower)
if (len(mutual_friend) == 0):
    print "There are no mutual friends."
else:
    mutual_table = PrettyTable(field_names=['Mutual Friend ID', 'Screen Name']) # Create table for mutual friend
    for mutual in mutual_friend:
        mutual_table.add_row([mutual[0], mutual[1]])
    mutual_table.align['Mutual Friend ID'], mutual_table.align['Screen Name'] = 'l', 'c'
    print mutual_table






There are no mutual friends.


*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [4]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

